In [1]:
import os
import re
import pandas as pd
import sqlite3
import configparser
import shlex
import subprocess
import json
import atexit
import gradio as gr
from langchain_mistralai import ChatMistralAI
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from typing import TypedDict, Dict, Any

# Определяем тип состояния
class GraphState(TypedDict):
    input: str
    last_path_vtune: str
    module_names: list
    last_path_csv: str
    last_path_callstack: str
    last_path_db: str
    result: str
    command: str
    args: str
    cleaned_args: str
    top_n: str

# Глобальные переменные
gradio_interface_instance = None

# Чтение API-ключа
check_path = os.path.dirname(os.getcwd())
config_path = os.path.join(check_path, "GPTPROF", "config.ini")
config = configparser.ConfigParser()
config.read(config_path)

try:
    mistral_api_key = config["API_KEYS"]["MISTRAL_API_KEY"]
except KeyError:
    raise ValueError("API key not found in config.ini")

# Инициализация модели
mistral_model = "codestral-latest"
llm = ChatMistralAI(model=mistral_model, temperature=0, api_key=mistral_api_key)

In [2]:
@tool
def export_vtune_to_csv(vtune_file_path: str) -> Dict[str, Any]:
    """
    Exports data from a .vtune file to a CSV file using Intel VTune.
    Returns a dictionary with the result message and the path to the exported CSV.

    Args:
        vtune_file_path (str): Path to the .vtune file.

    Returns:
        dict: Result message and path to the exported CSV.
    """
    if not vtune_file_path:
        return {"message": "Error: File path is empty", "csv_path": None}

    vtune_file_path = vtune_file_path.strip('"')

    if not vtune_file_path.endswith(".vtune"):
        return {"message": "Error: File must have .vtune extension", "csv_path": None}

    if not os.path.exists(vtune_file_path):
        return {"message": f"Error: File {vtune_file_path} not found", "csv_path": None}

    try:
        file_name = os.path.splitext(os.path.basename(vtune_file_path))[0]
        current_directory = os.getcwd()
        output_csv = os.path.join(current_directory, f"{file_name}.csv")
        command = (
            f'vtune -report hotspots -r {shlex.quote(vtune_file_path)} -format=csv -report-output {output_csv}'
        )

        subprocess.run(command, shell=True, check=True)
        return {"message": f"Data exported to {output_csv}", "csv_path": output_csv}

    except subprocess.CalledProcessError as e:
        return {"message": f"Export failed: {e}", "csv_path": None}

# Адаптер для узла графа
def export_vtune_node(state: GraphState) -> Dict[str, Any]:
    """
    Node that processes VTune export.
    Exports data from a .vtune file to CSV and returns updated state.

    Args:
        state (GraphState): Current state of the graph.

    Returns:
        dict: Updated state with the result of the export.
    """
    # Логируем входное состояние
    logging.info("Export node: Starting VTune export process.")
    vtune_path = state.get("last_path_vtune", "")
    logging.info(f"Export node: Received VTune file path: {vtune_path}")

    # Вызываем инструмент для экспорта
    tool_result = export_vtune_to_csv.invoke({"vtune_file_path": vtune_path})
    logging.info(f"Export node: Tool result - Message: {tool_result['message']}, CSV Path: {tool_result['csv_path']}")

    # Обновляем состояние
    updated_state = {
        **state,
        "input": "",
        "last_path_csv": tool_result["csv_path"],
        "result": tool_result["message"]
    }
    logging.info(f"Export node: Updated state after export: {updated_state}")

    return updated_state

In [3]:
@tool
def filter_by_module(module_names: list, csv_file_path: str) -> Dict[str, Any]:
    """
    Filters data by specified modules and saves the result in a new CSV file.
    Returns a dictionary with the result message and the path to the filtered CSV.

    Args:
        module_names (list): List of module names to filter by.
        csv_file_path (str): Path to the CSV file for filtering.

    Returns:
        dict: Result message and path to the filtered CSV.
    """
    if not os.path.exists(csv_file_path):
        return {"message": f"Error: File {csv_file_path} not found.", "filtered_csv_path": None}

    try:
        # Чтение CSV-файла
        df = pd.read_csv(csv_file_path, sep="\t")

        # Проверка наличия необходимых столбцов
        required_columns = {"Module", "Source File"}
        if not required_columns.issubset(df.columns):
            return {
                "message": f"Error: Missing required columns in CSV file. Expected {required_columns}, got {set(df.columns)}.",
                "filtered_csv_path": None
            }

        # Фильтрация данных
        filtered_df = df[
            (df["Module"].isin(module_names)) &
            (
                    (df["Source File"].str.endswith(".cpp") | df["Source File"].str.endswith(".hpp")) |
                    (df["Source File"] == "[Unknown]")
            )
            ]

        if filtered_df.empty:
            return {"message": "Error: No data found for the specified modules.", "filtered_csv_path": None}

        # Сохранение отфильтрованных данных
        file_name = os.path.basename(csv_file_path)
        filtered_csv_path = os.path.join(os.path.dirname(csv_file_path), f"filtered_{file_name}")
        filtered_df.to_csv(filtered_csv_path, sep="\t", index=False)

        return {
            "message": f"Data filtered by modules {module_names} and saved to {filtered_csv_path}.",
            "filtered_csv_path": filtered_csv_path
        }

    except Exception as e:
        return {"message": f"Error processing data: {e}", "filtered_csv_path": None}


def filter_by_module_node(state: GraphState) -> Dict[str, Any]:
    module_names = state.get("module_names", [])
    csv_file_path = state.get("last_path_csv", None)

    try:
        # Вызываем инструмент через invoke
        tool_result = filter_by_module.invoke({"module_names": module_names, "csv_file_path": csv_file_path})
    except Exception as e:
        return {
            **state,
            "input": "",
            "result": f"Error: Failed to call filter_by_module. Details: {e}"
        }

    updated_state = {
        **state,
        "input": "",
        "last_path_csv": tool_result["filtered_csv_path"],
        "result": tool_result["message"]
    }

    return updated_state

In [4]:
@tool
def filter_top_n_rows(top_n: str, last_path_csv: str) -> Dict[str, Any]:
    """
    Фильтрует топ-N строк из последнего экспортированного CSV-файла и сохраняет их в новый файл.

    Args:
        top_n (str): Количество строк для возврата.
        last_path_csv (str): Путь к последнему экспортированному CSV-файлу.

    Returns:
        dict: Результат выполнения (сообщение и путь к новому файлу).
    """
    # Проверка, что last_path_csv существует
    if not last_path_csv or not os.path.exists(last_path_csv):
        return {
            "message": "Ошибка: Нет данных для фильтрации. Сначала экспортируйте данные из .vtune.",
            "top_n_csv_path": None
        }

    try:
        # Преобразуем top_n в целое число
        n = int(top_n)
        if n <= 0:
            return {
                "message": "Ошибка: Параметр top_n должен быть положительным числом.",
                "top_n_csv_path": None
            }

        # Чтение данных из последнего экспортированного CSV-файла
        df = pd.read_csv(last_path_csv, sep="\t")

        # Получение топ-N строк
        top_n_rows = df.head(n)

        # Формирование имени файла с топ-N строками
        file_name = os.path.basename(last_path_csv)  # Имя файла с расширением
        file_name_without_ext = os.path.splitext(file_name)[0]  # Имя файла без расширения
        top_n_csv_path = os.path.join(os.path.dirname(last_path_csv), f"top_{n}_{file_name_without_ext}.csv")

        # Сохранение топ-N строк в новый файл
        top_n_rows.to_csv(top_n_csv_path, sep="\t", index=False)

        return {
            "message": f"Топ-{n} строк сохранены в {top_n_csv_path}.",
            "top_n_csv_path": top_n_csv_path
        }

    except ValueError:
        return {
            "message": "Ошибка: Параметр top_n должен быть целым числом.",
            "top_n_csv_path": None
        }
    except Exception as e:
        return {
            "message": f"Ошибка при обработке данных: {e}",
            "top_n_csv_path": None
        }
    
def filter_top_node(state: GraphState) -> Dict[str, Any]:
    """
    Node adapter for the filter_top_n_rows tool.
    Extracts top_n and last_path_csv from the state, calls the tool,
    and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after filtering top-N rows.
    """
    # Извлекаем параметры из состояния
    top_n = state.get("top_n", None)
    last_path_csv = state.get("last_path_csv", None)

    # Проверяем, что last_path_csv существует
    if not last_path_csv or not os.path.exists(last_path_csv):
        return {
            **state,
            "input": "",
            "result": "Ошибка: Нет данных для фильтрации. Сначала экспортируйте данные из .vtune."
        }

    try:
        # Вызываем инструмент
        tool_result = filter_top_n_rows.invoke({"top_n": top_n, "last_path_csv": last_path_csv})

        # Обновляем состояние
        updated_state = {
            **state,
            "input": "",
            "last_path_csv": tool_result["top_n_csv_path"],
            "result": tool_result["message"]
        }
        return updated_state

    except Exception as e:
        return {
            **state,
            "input": "",
            "result": f"Ошибка: Не удалось выполнить filter_top. Подробности: {e}"
        }

In [5]:
@tool
def save_path_files_to_db(directory: str) -> Dict[str, Any]:
    """
    Searches for all files with extensions .cpp, .h, and .hpp in the specified directory
    and saves their full paths to a database. The database name is generated from the last folder of the directory path.

    Args:
        directory (str): Directory to search for files. This is a required field.

    Returns:
        dict: Message indicating the result of the operation and the path to the database.
    """
    # Проверка существования директории
    if not os.path.exists(directory):
        return {
            "message": f"Error: Directory {directory} does not exist.",
            "db_path": None
        }

    # Генерация имени базы данных из последней папки в пути
    db_name = os.path.basename(os.path.normpath(directory)) + ".db"

    # Сохраняем базу данных в текущей рабочей директории
    current_directory = os.getcwd()  # Получаем текущую рабочую директорию
    db_path = os.path.join(current_directory, db_name)

    try:
        # Подключение к базе данных (или создание новой)
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Создание таблицы, если она не существует
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS files (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            file_name TEXT NOT NULL,
            file_path TEXT NOT NULL,
            namespace TEXT NOT NULL
        )
        """)
        conn.commit()

        # Поиск файлов с расширениями .cpp, .h и .hpp
        target_extensions = (".cpp", ".h", ".hpp")
        found_files = []
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith(target_extensions):
                    file_path = os.path.join(root, file)
                    namespace = os.path.basename(root)  # Имя папки, в которой лежит файл
                    found_files.append((file, file_path, namespace))

        # Вставка данных в базу данных
        for file_name, file_path, namespace in found_files:
            cursor.execute("""
            INSERT INTO files (file_name, file_path, namespace) VALUES (?, ?, ?)
            """, (file_name, file_path, namespace))
        conn.commit()
        conn.close()

        # Возвращаем результат
        return {
            "message": f"Found {len(found_files)} files. Data saved to database {db_path}.",
            "db_path": db_path
        }

    except Exception as e:
        return {
            "message": f"Error processing data: {e}",
            "db_path": None
        }

def save_paths_node(state: GraphState) -> Dict[str, Any]:
    """
    Node adapter for the save_path_files_to_db tool.
    Extracts directory from the state (from cleaned_args), calls the tool via invoke,
    and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after saving paths to the database.
    """
    # Извлекаем директорию из cleaned_args
    directory = state.get("cleaned_args", None)

    # Проверяем, что директория указана
    if not directory or not os.path.exists(directory):
        return {
            **state,
            "input": "",
            "result": f"Error: Directory {directory} does not exist or is invalid."
        }

    try:
        # Вызов инструмента
        tool_result = save_path_files_to_db.invoke({"directory": directory})

        # Обновляем состояние
        updated_state = {
            **state,
            "input": "",
            "last_path_db": tool_result["db_path"],  # Сохраняем путь к базе данных в новом поле
            "result": tool_result["message"]
        }
        return updated_state

    except Exception as e:
        return {
            **state,
            "input": "",
            "result": f"Error: Failed to execute save_paths. Details: {e}"
        }

In [6]:
def create_function_structure(row):
    """
    Creates a function structure based on a DataFrame row.

    Args:
        row (pd.Series): A row from the DataFrame.

    Returns:
        dict: A dictionary representing the function structure.
    """
    # Полное имя функции (без изменений)
    full_function_name = row["Function"]

    # Разбиваем строку на части
    parts = full_function_name.split("::", maxsplit=2)
    folder_name = parts[0].lower()
    namespace = parts[1]
    function_name = parts[2]

    # Убираем всё, что в скобках, включая сами скобки
    function_name = re.sub(r"\(.*?\)", "", function_name).strip()

    # Если function_name содержит шаблонные параметры, оставляем только имя функции
    if "<" in function_name:
        function_name = function_name.split("::")[-1]

    source_file = row["Source File"]
    module = row["Module"]

    # Создание структуры
    function_structure = {
        "folder_name": folder_name,
        "full_folder_name": "",
        "namespace": namespace,
        "function_name": function_name,
        "full_function_name": full_function_name,
        "source_file": source_file,
        "module": module,
        "return_type": "",
        "arguments": "",
        "code": ""
    }
    return function_structure


def find_and_set_full_folder_name(function: dict, db_name: str = None) -> None:
    """
    Finds the full path to the source code file in the database and adds it to the function structure.

    Args:
        function (dict): A dictionary representing the function structure.
        db_name (str, optional): The name of the database created by the save_path_files_to_db tool. 
                                 If not provided, the global variable path_last_db is used.
    """
    # Если имя базы данных не указано, используем глобальную переменную path_last_db
    if db_name is None:
        global path_last_db
        if path_last_db is None:
            print("Error: Database path is not specified and path_last_db is not set.")
            return
        db_name = path_last_db

    folder_name = function["folder_name"]
    source_file = function["source_file"]

    # Подключение к базе данных
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Поиск полного пути к файлу в базе данных
    cursor.execute("""
    SELECT file_path FROM files
    WHERE file_name = ? AND namespace = ?
    """, (source_file, folder_name))

    result = cursor.fetchone()
    conn.close()

    # Если путь найден, добавляем его в структуру функции
    if result:
        function["full_folder_name"] = result[0]
    else:
        print(f"File {source_file} in folder {folder_name} not found in database {db_name}.")


def extract_function_details(function):
    """
    Extracts the function signature (return type, arguments) and full code from the source file.

    Args:
        function (dict): A dictionary representing the function structure.
    """
    # Формирование полного пути к файлу
    file_path = function["full_folder_name"]

    # Чтение содержимого файла
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return

    function_name = function["function_name"]

    # Регулярное выражение для поиска функции (включая возвращаемый тип и аргументы)
    function_pattern = re.compile(rf"(.*?)\b{function_name}\s*\(([^)]*)\)\s*{{")

    # Поиск функции
    function_match = function_pattern.search(content)
    if not function_match:
        print(f"Function {function_name} not found in file {file_path}.")
        return

    # Извлечение return_type и arguments из совпадения
    return_type = function_match.group(1).strip()  # Возвращаемый тип
    arguments = function_match.group(2).strip()   # Аргументы

    # Начальная позиция функции (после сигнатуры)
    start_pos = function_match.end()

    # Счётчик скобок
    brace_count = 1  # Начинаем с 1, так как первая скобка уже найдена
    function_code = function_match.group(0)  # Начинаем с сигнатуры функции
    i = start_pos

    # Итерация по символам для поиска конца функции
    while i < len(content):
        if content[i] == "{":
            brace_count += 1
        elif content[i] == "}":
            brace_count -= 1

        function_code += content[i]

        # Если все скобки закрыты, завершаем чтение
        if brace_count == 0:
            break

        i += 1

    # Заполнение полей return_type, arguments и code
    function["return_type"] = return_type
    function["arguments"] = arguments
    function["code"] = function_code


def save_to_database(functions_list, db_name):
    """
    Saves the data about functions to an SQLite database.
    Args:
        functions_list (list): A list of function structures.
        db_name (str): The name of the database.
    """
    # Подключение к базе данных (или создание новой)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Создание таблицы с добавлением поля full_function_name
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS functions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        folder_name TEXT NOT NULL,
        full_folder_name TEXT NOT NULL,
        namespace TEXT NOT NULL,
        function_name TEXT NOT NULL,
        full_function_name TEXT NOT NULL,
        source_file TEXT NOT NULL,
        module TEXT NOT NULL,
        return_type TEXT,
        arguments TEXT,
        code TEXT
    )
    """)
    conn.commit()

    # Вставка данных с учетом full_function_name
    for function in functions_list:
        cursor.execute("""
        INSERT INTO functions (
            folder_name, full_folder_name, namespace, function_name, full_function_name,
            source_file, module, return_type, arguments, code
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            function["folder_name"],
            function["full_folder_name"],
            function["namespace"],
            function["function_name"],
            function["full_function_name"],
            function["source_file"],
            function["module"],
            function["return_type"],
            function["arguments"],
            function["code"]
        ))

    conn.commit()
    conn.close()

@tool
def process_functions_and_save_to_db(input_csv_path: str, db_name: str) -> Dict[str, Any]:
    """
    Processes a CSV file, extracts information about functions, finds source files,
    extracts function code, and saves the data to a database.

    Args:
        input_csv_path (str): Path to the CSV file.
        db_name (str): Name of the database to save the data.

    Returns:
        dict: Message indicating the result of the operation and the path to the database.
    """
    logging.info(f"Process functions tool: Received input_csv_path: {input_csv_path}")
    logging.info(f"Process functions tool: Received db_name: {db_name}")

    # Проверка существования файла CSV
    if not os.path.exists(input_csv_path):
        logging.error(f"Process functions tool: File {input_csv_path} does not exist.")
        return {
            "message": f"Error: File {input_csv_path} does not exist.",
            "db_path": None
        }

    try:
        # Чтение CSV-файла
        logging.info("Process functions tool: Reading CSV file.")
        df = pd.read_csv(input_csv_path, sep="\t")

        # Создание списка структур функций
        logging.info("Process functions tool: Creating function structures.")
        functions_list = []
        for index, row in df.iterrows():
            function_structure = create_function_structure(row)
            functions_list.append(function_structure)
            logging.debug(f"Process functions tool: Created function structure: {function_structure}")

        # Поиск полных путей к исходным файлам
        logging.info("Process functions tool: Finding full paths to source files.")
        for function in functions_list:
            find_and_set_full_folder_name(function, db_name)
            logging.debug(f"Process functions tool: Found full folder name: {function['full_folder_name']}")

        # Извлечение деталей функции
        logging.info("Process functions tool: Extracting function details.")
        for function in functions_list:
            extract_function_details(function)
            logging.debug(f"Process functions tool: Extracted function details: {function}")

        # Сохранение данных в базу данных
        logging.info("Process functions tool: Saving data to database.")
        save_to_database(functions_list, db_name)

        logging.info(f"Process functions tool: Data successfully processed and saved to database {db_name}.")
        return {
            "message": f"Data successfully processed and saved to database {db_name}.",
            "db_path": db_name
        }

    except Exception as e:
        logging.error(f"Process functions tool: Error processing data: {e}")
        return {
            "message": f"Error processing data: {e}",
            "db_path": None
        }

def process_functions_node(state: GraphState) -> Dict[str, Any]:
    """
    Node adapter for the process_functions_and_save_to_db tool.
    Extracts input_csv_path and db_name from the state, calls the tool via invoke,
    and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after processing functions and saving to the database.
    """
    logging.info("Process functions node: Entering process_functions_node function.")
    logging.info(f"Process functions node: Received state: {state}")

    # Извлекаем входные данные из состояния
    input_csv_path = state.get("last_path_csv", None)
    db_name = state.get("last_path_db", None)  # Используем новое поле last_path_db

    logging.info(f"Process functions node: Extracted input_csv_path: {input_csv_path}")
    logging.info(f"Process functions node: Extracted db_name: {db_name}")

    # Проверяем, что CSV-файл указан
    if not input_csv_path or not os.path.exists(input_csv_path):
        logging.error(f"Process functions node: CSV file {input_csv_path} does not exist.")
        return {
            **state,
            "input": "",
            "result": f"Error: CSV file {input_csv_path} does not exist."
        }

    # Проверяем, что база данных указана
    if not db_name or not os.path.exists(db_name):
        logging.error(f"Process functions node: Database {db_name} does not exist.")
        return {
            **state,
            "input": "",
            "result": f"Error: Database {db_name} does not exist."
        }

    try:
        logging.info("Process functions node: Invoking process_functions_and_save_to_db tool.")
        tool_result = process_functions_and_save_to_db.invoke({
            "input_csv_path": input_csv_path,
            "db_name": db_name
        })
        logging.info(f"Process functions node: Tool result: {tool_result}")

        # Обновляем состояние
        updated_state = {
            **state,
            "input": "",
            "last_path_db": tool_result["db_path"],  # Обновляем путь к базе данных
            "result": tool_result["message"]
        }
        logging.info(f"Process functions node: Updated state: {updated_state}")
        return updated_state

    except Exception as e:
        logging.error(f"Process functions node: Error executing process_functions_and_save_to_db: {e}")
        return {
            **state,
            "input": "",
            "result": f"Error: Failed to execute process_functions. Details: {e}"
        }

In [7]:
@tool
def show_top_rows_from_db(count_rows: int, db_name: str) -> Dict[str, Any]:
    """
    Fetches the top N rows from the 'functions' table in the SQLite database and returns them as a formatted string.

    Args:
        count_rows (int): Number of rows to fetch.
        db_name (str): Path to the SQLite database.

    Returns:
        dict: Message indicating the result of the operation and the fetched data.
    """
    # Проверка существования файла базы данных
    if not os.path.exists(db_name):
        return {
            "message": f"Error: Database {db_name} does not exist.",
            "data": None
        }

    try:
        # Подключение к базе данных
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Выполнение SQL-запроса для выборки первых N строк
        cursor.execute(
            f"SELECT id, full_folder_name, full_function_name, code FROM functions LIMIT {count_rows}"
        )
        rows = cursor.fetchall()

        # Закрытие соединения с базой данных
        conn.close()

        # Если данные найдены, форматируем их построчно
        if rows:
            formatted_output = []
            for row in rows:
                id, full_folder_name, full_function_name, code = row

                # Форматируем содержимое столбца `code` как обычный текст
                if isinstance(code, str):
                    formatted_code = code.replace("\\n", "\n")  # Замена \n на реальные переносы строк
                else:
                    formatted_code = "No code available."

                # Формируем строку без Markdown
                formatted_row = (
                    f"ID: {id}\n"
                    f"Full Folder Name: {full_folder_name}\n"
                    f"Function Name: {full_function_name}\n"  # Добавляем отображение function_name
                    f"Code:\n{formatted_code}\n"
                    f"---\n"  # Разделитель между строками
                )
                formatted_output.append(formatted_row)

            # Объединяем все строки в один вывод
            message = (
                    f"Fetched top {count_rows} rows from database `{db_name}`.\n\n"
                    + "\n".join(formatted_output)
            )
            return {
                "message": message,
                "data": rows
            }
        else:
            return {
                "message": "No data found in the database.",
                "data": None
            }

    except sqlite3.Error as e:
        return {
            "message": f"Error working with the database: {e}",
            "data": None
        }

def show_db_node(state: GraphState) -> Dict[str, Any]:
    """
    Node adapter for the show_top_rows_from_db tool.
    Extracts count_rows and last_path_db from the state, calls the tool via invoke,
    and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after fetching data from the database.
    """
    logging.info("Show DB node: Entering show_db_node function.")
    logging.info(f"Show DB node: Received state: {state}")

    # Извлекаем параметры из состояния
    count_rows = state.get("cleaned_args", None)
    db_name = state.get("last_path_db", None)

    logging.info(f"Show DB node: Extracted count_rows: {count_rows}")
    logging.info(f"Show DB node: Extracted db_name: {db_name}")

    # Проверяем, что база данных указана
    if not db_name or not os.path.exists(db_name):
        logging.error(f"Show DB node: Database {db_name} does not exist.")
        return {
            **state,
            "input": "",
            "result": f"Error: Database {db_name} does not exist."
        }

    # Проверяем, что count_rows указан и является положительным целым числом
    if not isinstance(count_rows, int) or count_rows <= 0:
        logging.error(f"Show DB node: Invalid count_rows value: {count_rows}")
        return {
            **state,
            "input": "",
            "result": "Error: Invalid count_rows value. Please provide a positive integer."
        }

    try:
        logging.info("Show DB node: Invoking show_top_rows_from_db tool.")
        tool_result = show_top_rows_from_db.invoke({"count_rows": count_rows, "db_name": db_name})
        logging.info(f"Show DB node: Tool result: {tool_result}")

        # Обновляем состояние
        updated_state = {
            **state,
            "input": "",
            "result": tool_result["message"],
            "output_data": tool_result["data"]  # Сохраняем данные для отображения
        }
        logging.info(f"Show DB node: Updated state: {updated_state}")

        return updated_state

    except Exception as e:
        logging.error(f"Show DB node: Error executing show_top_rows_from_db: {e}")
        return {
            **state,
            "input": "",
            "result": f"Error: Failed to execute show_db. Details: {e}"
        }

In [8]:
def get_function_name_from_db(function_id: int, db_name: str) -> str:
    """
    Retrieves the full function name from the database based on its ID.

    Args:
        function_id (int): ID of the function in the database.
        db_name (str): Path to the database.

    Returns:
        str: Full name of the function, or an empty string if not found.
    """
    if not db_name or not os.path.exists(db_name):
        logging.error(f"Database file {db_name} not found.")
        return ""

    try:
        logging.info(f"Connecting to database: {db_name}")
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Запрос для получения полного имени функции по её ID
        logging.info(f"Retrieving function with ID {function_id} from the database.")
        cursor.execute("""
        SELECT full_function_name FROM functions
        WHERE id = ?
        """, (function_id,))

        result = cursor.fetchone()
        conn.close()

        if result:
            logging.info(f"Function with ID {function_id} found: {result[0]}")
            return result[0]
        else:
            logging.warning(f"Function with ID {function_id} not found in the database.")
            return ""

    except Exception as e:
        logging.error(f"Error retrieving function name from database: {e}")
        return ""

@tool
def export_callstack_to_csv(
        vtune_file_path: str,
        function_id: int,
        db_path: str,
        row_limit: str = "100000"  # Лимит строк по умолчанию
) -> str:
    """
    Exports the callstack of a specific function from a .vtune file to a CSV file using Intel VTune.
    The function is identified by its ID in the database.

    Args:
        vtune_file_path (str): Path to the .vtune file.
        function_id (int): ID of the function in the database.
        db_path (str): Path to the database.
        row_limit (str, optional): Maximum number of rows to export. Default is "100000".

    Returns:
        str: Message indicating the result of the operation.
    """
    logging.info("Starting export_callstack_to_csv tool.")

    # Проверка входных данных
    if not vtune_file_path:
        logging.error("VTune file path is required.")
        return "Error: VTune file path is required."
    if not vtune_file_path.endswith(".vtune"):
        logging.error(f"File must have .vtune extension. Provided: {vtune_file_path}")
        return "Error: File must have .vtune extension."
    if not os.path.exists(vtune_file_path):
        logging.error(f"File {vtune_file_path} not found.")
        return f"Error: File {vtune_file_path} not found."

    if not db_path or not os.path.exists(db_path):
        logging.error(f"Database file {db_path} not found.")
        return f"Error: Database file {db_path} not found."

    # Получение полного имени функции из базы данных по её ID
    logging.info(f"Retrieving function name for ID {function_id} from database.")
    full_function_name = get_function_name_from_db(function_id, db_path)
    if not full_function_name:
        logging.error(f"Function with ID {function_id} not found in the database.")
        return f"Error: Function with ID {function_id} not found in the database."

    try:
        # Формирование имени выходного файла
        file_name = os.path.splitext(os.path.basename(vtune_file_path))[0]
        safe_function_name = full_function_name.replace("::", "_").replace(" ", "_")  # Безопасное имя функции
        output_csv = f"{file_name}_callstack_{safe_function_name}_{function_id}.csv"
        logging.info(f"Output CSV file will be created: {output_csv}")

        # Команда для экспорта callstacks
        command = (
            f'vtune -R callstacks '
            f'-r "{shlex.quote(vtune_file_path)}" '
            f'-report-output "{output_csv}" '
            f'-format csv '
            f'-csv-delimiter comma '
            f'-filter "function={full_function_name}" '
            f'-limit {row_limit}'
        )
        logging.info(f"Executing VTune command: {command}")

        # Выполнение команды
        subprocess.run(command, shell=True, check=True)

        logging.info(f"Callstack data for function '{full_function_name}' exported to {output_csv}")
        return f"Callstack data for function '{full_function_name}' exported to {output_csv}"

    except subprocess.CalledProcessError as e:
        logging.error(f"Export failed: {e}")
        return f"Export failed: {e}"

def export_callstack_node(state: GraphState) -> GraphState:
    """
    Node adapter for the export_callstack_to_csv tool.
    Extracts parameters from cleaned_args and state, calls the tool using invoke, and updates the state.

    Args:
        state (GraphState): Current state of the graph.

    Returns:
        GraphState: Updated state after exporting callstack data.
    """
    logging.info("Starting export_callstack_node.")

    # Извлекаем cleaned_args из состояния
    cleaned_args = state.get("cleaned_args", "")
    last_path_vtune = state.get("last_path_vtune", None)
    last_path_db = state.get("last_path_db", None)

    # Разбираем cleaned_args на отдельные параметры
    args_list = cleaned_args.split()
    if len(args_list) < 1:
        logging.error("Insufficient arguments for export_callstack.")
        return {
            **state,
            "result": "Error: Insufficient arguments for export_callstack.",
        }

    # Извлекаем обязательные параметры
    function_id = int(args_list[0])  # Первый параметр - function_id

    # Инициализируем необязательные параметры значениями по умолчанию
    row_limit = args_list[1] if len(args_list) > 1 else "100000"  # Лимит строк по умолчанию

    # Проверяем обязательные параметры
    if not last_path_vtune:
        logging.error("VTune file path is required (last_path_vtune).")
        return {
            **state,
            "result": "Error: VTune file path is required (last_path_vtune).",
        }
    if not last_path_db:
        logging.error("Database file path is required (last_path_db).")
        return {
            **state,
            "result": "Error: Database file path is required (last_path_db).",
        }
    if function_id is None:
        logging.error("Function ID is required.")
        return {
            **state,
            "result": "Error: Function ID is required.",
        }

    # Вызываем тул для экспорта callstack с помощью метода invoke
    try:
        logging.info(f"Invoking export_callstack_to_csv with function_id={function_id}, row_limit={row_limit}.")
        tool_result = export_callstack_to_csv.invoke(
            {
                "vtune_file_path": last_path_vtune,
                "function_id": function_id,
                "db_path": last_path_db,
                "row_limit": row_limit
            }
        )
        logging.info(f"Tool executed successfully: {tool_result}")
    except Exception as e:
        logging.error(f"Error executing export_callstack_to_csv: {e}")
        return {
            **state,
            "result": f"Error executing export_callstack_to_csv: {e}",
        }

    # Возвращаем обновленное состояние
    return {
        **state,
        "result": tool_result,  # Результат работы тула
        "last_path_callstack": tool_result.split("exported to ")[-1].strip(),  # Сохраняем путь в новое поле
    }

In [9]:
@tool
def filter_callstack_to_csv(
        input_csv_path: str,
        call_number_limit: int,
        subcall_number_limit: int,
        output_folder: str = None
) -> str:
    """
    Filters the callstack data based on call_number_limit and subcall_number_limit.
    Saves the filtered data to a new CSV file.

    Args:
        input_csv_path (str): Path to the input CSV file.
        call_number_limit (int): Maximum number of top-level calls to include.
        subcall_number_limit (int): Maximum number of subcalls for each top-level call.
        output_folder (str, optional): Folder to save the filtered CSV file. Defaults to the folder of the input file.

    Returns:
        str: Message indicating the result of the operation and the path to the filtered file.
    """
    logging.info("Starting filter_callstack_to_csv tool.")

    # Проверка существования входного файла
    if not os.path.exists(input_csv_path):
        logging.error(f"Input file {input_csv_path} does not exist.")
        return f"Error: Input file {input_csv_path} does not exist."

    try:
        # Чтение CSV-файла
        with open(input_csv_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # Формирование имени выходного файла
        file_name = os.path.splitext(os.path.basename(input_csv_path))[0]
        output_csv = f"filtered_{call_number_limit}_{subcall_number_limit}_{file_name}.csv"
        if output_folder:
            output_csv = os.path.join(output_folder, output_csv)
        else:
            output_csv = os.path.join(os.path.dirname(input_csv_path), output_csv)

        # Инициализация результата
        result_lines = []
        result_lines.append(lines[0].strip())  # Заголовок
        result_lines.append(lines[1].strip())
        # Разделение на блоки функций
        function_blocks = []
        current_block = []

        for line in lines[2:]:
            stripped_line = line.strip()
            if not stripped_line:
                continue  # Пропускаем пустые строки

            # Разделяем строку по запятым
            parts = stripped_line.split(",")
            if len(parts) < 3:
                continue  # Пропускаем строки без достаточного количества полей

            cpu_time = parts[2].strip()  # CPU Time находится в третьем столбце
            if cpu_time and float(cpu_time) > 0:
                # Если текущая строка имеет ненулевое время, это начало нового блока
                if current_block:
                    function_blocks.append(current_block)
                current_block = [stripped_line]
            else:
                # Добавляем строку в текущий блок
                current_block.append(stripped_line)

        # Добавляем последний блок, если он есть
        if current_block:
            function_blocks.append(current_block)

        # Логирование найденных блоков
        for i, block in enumerate(function_blocks, start=1):
            logging.debug(f"Found block #{i}:")
            logging.debug("\n".join(block))

        # Обработка блоков функций
        call_count = 0
        for block in function_blocks:
            if call_count >= call_number_limit:
                break  # Ограничение на количество функций

            logging.info(f"Processing block #{call_count + 1}:")
            logging.info("\n".join(block))  # Отображение содержимого блока

            # Добавляем строки из блока с учетом лимита подвызовов
            subcall_count = 0
            for subcall in block:
                if subcall_count >= subcall_number_limit:
                    break  # Ограничение на количество подвызовов
                result_lines.append(subcall)
                subcall_count += 1

            call_count += 1

        # Запись результата в новый CSV-файл
        with open(output_csv, "w", encoding="utf-8") as file:
            file.write("\n".join(result_lines))

        logging.info(f"Filtered data saved to {output_csv}")
        return f"Filtered callstack data saved to {output_csv}"

    except Exception as e:
        logging.error(f"Error filtering callstack data: {e}")
        return f"Error: Failed to filter callstack data. Details: {e}"

def filter_callstack_node(state: GraphState) -> GraphState:
    """
    Node adapter for the filter_callstack_to_csv tool.
    Extracts parameters from cleaned_args and state, calls the tool using invoke, and updates the state.

    Args:
        state (GraphState): Current state of the graph.

    Returns:
        GraphState: Updated state after filtering callstack data.
    """
    logging.info("Starting filter_callstack_node.")

    # Извлекаем cleaned_args из состояния
    cleaned_args = state.get("cleaned_args", "")
    last_path_callstack = state.get("last_path_callstack", None)

    # Разбираем cleaned_args на отдельные параметры
    args_list = cleaned_args.split()
    if len(args_list) < 2:
        logging.error("Insufficient arguments for filter_callstack.")
        return {
            **state,
            "result": "Error: Insufficient arguments for filter_callstack. Expected format: 'call_number_limit subcall_number_limit'.",
        }

    # Извлекаем обязательные параметры
    try:
        call_number_limit = int(args_list[0])  # Первый параметр - call_number_limit
        subcall_number_limit = int(args_list[1])  # Второй параметр - subcall_number_limit
    except ValueError:
        logging.error("Invalid arguments for filter_callstack. Both call_number_limit and subcall_number_limit must be integers.")
        return {
            **state,
            "result": "Error: Invalid arguments for filter_callstack. Both call_number_limit and subcall_number_limit must be integers.",
        }

    # Проверяем обязательные параметры
    if not last_path_callstack or not os.path.exists(last_path_callstack):
        logging.error("Callstack file path is required (last_path_callstack).")
        return {
            **state,
            "result": "Error: Callstack file path is required (last_path_callstack).",
        }

    # Вызываем тул для фильтрации стека вызовов
    try:
        logging.info(f"Invoking filter_callstack_to_csv with call_number_limit={call_number_limit}, subcall_number_limit={subcall_number_limit}.")
        tool_result = filter_callstack_to_csv.invoke(
            {
                "input_csv_path": last_path_callstack,
                "call_number_limit": call_number_limit,
                "subcall_number_limit": subcall_number_limit
            }
        )
        logging.info(f"Tool executed successfully: {tool_result}")
    except Exception as e:
        logging.error(f"Error executing filter_callstack_to_csv: {e}")
        return {
            **state,
            "result": f"Error executing filter_callstack_to_csv: {e}",
        }

    # Возвращаем обновленное состояние
    return {
        **state,
        "result": tool_result,  # Результат работы тула
        "last_path_callstack": tool_result.split("saved to ")[-1].strip(),  # Сохраняем путь в новое поле
    }

In [10]:
@tool
def save_callstack(path_db: str, path_callstack: str) -> str:
    """
    Saves callstack dependencies to the database.
    Creates the 'callstacks' table if it does not exist and populates it with data from the callstack file.

    Args:
        path_db (str): Path to the database file.
        path_callstack (str): Path to the callstack CSV file.

    Returns:
        str: Message indicating the result of the operation.
    """
    logging.info("Starting save_callstack tool.")

    # Проверка существования файлов
    if not path_db or not os.path.exists(path_db):
        logging.error("Database file is not specified or does not exist.")
        return "Error: Database file is not specified or does not exist."

    if not path_callstack or not os.path.exists(path_callstack):
        logging.error("Callstack file is not specified or does not exist.")
        return "Error: Callstack file is not specified or does not exist."

    try:
        # Подключение к базе данных
        conn = sqlite3.connect(path_db)
        cursor = conn.cursor()

        # Создание таблицы callstacks, если она не существует
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS callstacks (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                id_function INTEGER NOT NULL,
                full_func_name TEXT NOT NULL,
                FOREIGN KEY (id_function) REFERENCES functions(id)
            )
        """)
        conn.commit()

        # Чтение CSV-файла с callstack
        with open(path_callstack, "r", encoding="utf-8") as file:
            lines = file.readlines()

        # Первая строка содержит основную функцию
        header = lines[1].strip().split(",")
        main_function_name = header[0]  # Значение из столбца "Function"

        # Поиск id_function в таблице functions
        cursor.execute("SELECT id FROM functions WHERE full_function_name = ?", (main_function_name,))
        result = cursor.fetchone()
        if not result:
            logging.error(f"Function '{main_function_name}' not found in the 'functions' table.")
            return f"Error: Function '{main_function_name}' not found in the 'functions' table."

        id_function = result[0]

        # Заполнение таблицы callstacks
        for line in lines[1:]:
            stripped_line = line.strip()
            if not stripped_line:
                continue  # Пропускаем пустые строки

            parts = stripped_line.split(",")
            if len(parts) < 2:
                continue  # Пропускаем строки без достаточного количества полей

            function_stack = parts[1].strip()  # Значение из столбца "Function Stack"
            if function_stack:
                cursor.execute(
                    "INSERT INTO callstacks (id_function, full_func_name) VALUES (?, ?)",
                    (id_function, function_stack)
                )

        conn.commit()
        conn.close()

        logging.info("Callstack dependencies saved to the database.")
        return f"Callstack dependencies successfully saved to {path_db}."

    except Exception as e:
        logging.error(f"Error saving callstack dependencies: {e}")
        return f"Error: Failed to save callstack dependencies. Details: {e}"

def save_callstack_node(state: GraphState) -> GraphState:
    """
    Node adapter for the save_callstack tool.
    Extracts parameters from the state, calls the tool via invoke, and returns the updated state.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state after saving callstack data.
    """
    logging.info("Entering save_callstack_node function.")
    logging.info(f"Received state: {state}")

    # Извлечение параметров из состояния
    path_db = state.get("last_path_db", None)
    path_callstack = state.get("last_path_callstack", None)

    # Проверка обязательных параметров
    if not path_db or not os.path.exists(path_db):
        logging.error("Database path is not specified or does not exist.")
        return {
            **state,
            "input": "",
            "result": "Error: Database path is not specified or does not exist."
        }

    if not path_callstack or not os.path.exists(path_callstack):
        logging.error("Callstack file path is not specified or does not exist.")
        return {
            **state,
            "input": "",
            "result": "Error: Callstack file path is not specified or does not exist."
        }

    try:
        # Вызов тула через invoke
        logging.info("Invoking save_callstack tool via invoke.")
        tool_result = save_callstack.invoke({"path_db": path_db, "path_callstack": path_callstack})
        logging.info(f"Tool result: {tool_result}")

        # Обновление состояния
        updated_state = {
            **state,
            "input": "",
            "result": tool_result  # Результат работы тула
        }

        return updated_state

    except Exception as e:
        logging.error(f"Error executing save_callstack tool: {e}")
        return {
            **state,
            "input": "",
            "result": f"Error executing save_callstack tool: {e}"
        }

In [ ]:
import sqlite3

def print_callstack_table(db_name: str) -> None:
    """
    Debugging function to print the entire content of the 'callstacks' table to the console.

    Args:
        db_name (str): Path to the database file.
    """
    # Проверка существования файла базы данных
    if not db_name or not os.path.exists(db_name):
        print(f"Error: Database file {db_name} not found.")
        return

    try:
        # Подключение к базе данных
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Проверка существования таблицы callstacks
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='callstacks';")
        table_exists = cursor.fetchone()
        if not table_exists:
            print("Error: Table 'callstacks' does not exist in the database.")
            conn.close()
            return

        # Выборка всех данных из таблицы callstacks
        cursor.execute("SELECT * FROM callstacks;")
        rows = cursor.fetchall()

        # Получение заголовков столбцов
        column_names = [description[0] for description in cursor.description]

        # Вывод заголовков
        print("\t".join(column_names))

        # Вывод строк таблицы
        for row in rows:
            print("\t".join(map(str, row)))

        # Закрытие соединения
        conn.close()

    except Exception as e:
        print(f"Error printing 'callstacks' table: {e}")

print_callstack_table("E:\\AAStudy\\0_work\\Work\\Chukanov\\MyWork\\Diplom\\GPTPROF\\openmw.db")

In [11]:
def parse_command_node(state: GraphState) -> Dict[str, Any]:
    """
    Parses the user input and updates the state with parsed data.

    Args:
        state (dict): Current state of the graph.

    Returns:
        dict: Updated state with parsed data.
    """
    # Извлекаем входной запрос пользователя
    input_str = state.get("input", "").strip()

    if not input_str:
        return {
            "message": "Error: Empty input.",
            "state": state
        }

    # Разделяем команду на части
    parts = input_str.split(maxsplit=1)
    command = parts[0].lower()  # Команда (например, export, filter, filter_top)
    args = parts[1] if len(parts) > 1 else None  # Аргументы

    # Обработка команды export
    if command == "export":
        vtune_file_path = args.strip() if args else None
        return {
            **state,
            "command": "export",
            "args": args,
            "cleaned_args": vtune_file_path,
            "last_path_vtune": vtune_file_path
        }

    # Обработка команды filter
    elif command == "filter":
        module_names = [arg.strip() for arg in args.split(",") if arg.strip()] if args else []
        return {
            **state,
            "command": "filter",
            "args": args,
            "cleaned_args": module_names,
            "module_names": module_names
        }

    # Обработка команды filter_top
    elif command == "filter_top":
        top_n = args.strip() if args else None
        return {
            **state,
            "command": "filter_top",
            "args": args,
            "cleaned_args": top_n,
            "top_n": top_n
        }
    
    elif command == "save_paths":
        directory = args.strip() if args else None
        return {
            **state,
            "command": "save_paths",
            "args": args,
            "cleaned_args": directory,
            "directory": directory
        }
    
    elif command == "process_functions":
        return {
            **state,
            "command": "process_functions",
            "args": args,
            "cleaned_args": args
        }
    
    elif command == "show_db":
        try:
            count_rows = int(args.strip()) if args else None
            return {
                **state,
                "command": "show_db",
                "args": args,
                "cleaned_args": count_rows
            }
        except ValueError:
            return {
                **state,
                "command": "show_db",
                "args": args,
                "cleaned_args": None
            }

    elif command == "export_callstack":
        cleaned_args = args.strip() if args else ""
        return {
            **state,
            "command": "export_callstack",
            "args": args,
            "cleaned_args": cleaned_args
        }
    
    elif command == "filter_callstack":
        cleaned_args = args.strip() if args else ""
        return {
            **state,
            "command": "filter_callstack",
            "args": args,
            "cleaned_args": cleaned_args
        }
    
    elif command == "save_callstack":
        cleaned_args = args.strip() if args else ""
        return {
            **state,
            "command": "save_callstack",
            "args": args,
            "cleaned_args": cleaned_args
        }

    # Если команда не распознана
    else:
        return {
            **state,
            "command": "unknown",
            "args": args,
            "cleaned_args": None
        }

In [12]:
# Создаем граф
workflow = StateGraph(GraphState)

# Добавляем узлы
workflow.add_node("parse", parse_command_node)
workflow.add_node("export", export_vtune_node)
workflow.add_node("filter", filter_by_module_node)
workflow.add_node("filter_top", filter_top_node)
workflow.add_node("save_paths", save_paths_node)
workflow.add_node("process_functions", process_functions_node)
workflow.add_node("show_db", show_db_node)
workflow.add_node("export_callstack", export_callstack_node)
workflow.add_node("filter_callstack", filter_callstack_node)
workflow.add_node("save_callstack", save_callstack_node)

# Добавляем рёбра
workflow.add_edge(START, "parse")

# Большой условный переход после парсера
def conditional_edge(state: GraphState) -> str:
    """
    Determines the next node based on the parsed command.

    Args:
        state (GraphState): Current state of the graph.

    Returns:
        str: Name of the next node.
    """
    logging.info(f"Conditional edge: state in input '{state}'")
    command = state.get("command", "")
    logging.info(f"Conditional edge: Received command '{command}' from state.")

    if command == "export":
        return "export"
    elif command == "filter":
        return "filter"
    elif command == "filter_top":
        return "filter_top"
    elif command == "save_paths":
        return "save_paths"
    elif command == "process_functions":
        return "process_functions"
    elif command == "show_db":
        return "show_db"
    elif command == "export_callstack":
        return "export_callstack"
    elif command == "filter_callstack":
        return "filter_callstack"
    elif command == "save_callstack":
        return "save_callstack"
    elif command == "exit":
        return END
    else:
        logging.warning(f"Conditional edge: Unknown command '{command}'. Moving to END.")
        return END

workflow.add_conditional_edges("parse", conditional_edge)

# Добавляем рёбра к концу
workflow.add_edge("export", END)
workflow.add_edge("filter", END)
workflow.add_edge("filter_top", END)
workflow.add_edge("save_paths", END)
workflow.add_edge("process_functions", END)
workflow.add_edge("show_db", END)
workflow.add_edge("export_callstack", END)
workflow.add_edge("filter_callstack", END)
workflow.add_edge("save_callstack", END)


# Компилируем граф
compiled_graph = workflow.compile()

# Gradio интерфейс
import logging

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Уровень логирования (INFO для основных сообщений)
    format="%(asctime)s - %(levelname)s - %(message)s",  # Формат логов
    handlers=[
        logging.StreamHandler(),  # Вывод в консоль
        logging.FileHandler("codestral.log")  # Сохранение в файл
    ]
)

# Gradio интерфейс
def ask_codestral(user_input: str, session_state: dict = None):
    """
    Processes user input and updates the state using the compiled graph.
    Handles the 'exit' command to terminate the Gradio interface.

    Args:
        user_input (str): User input string.
        session_state (dict, optional): Current state of the session.

    Returns:
        tuple: Result message and updated state.
    """
    # Логируем входные данные
    logging.info(f"Received user input: {user_input}")
    if session_state:
        if session_state["input"] == "":
            session_state["input"] = user_input
    logging.info(f"Session state before processing: {session_state}")
    
    
    # Инициализируем состояние
    # state = session_state or {
    #     "input": user_input,
    #     "result": "",
    #     "last_path_vtune": None,
    #     "last_path_csv": None,
    #     "last_path_db": None,
    #     "module_names": [],
    #     "command": "",
    #     "args": "",
    #     "cleaned_args": "",
    #     "top_n": ""
    # }

    state = session_state or {
        "input": user_input,
        "result": "",
        "last_path_vtune": "E:/openmw/OMW_2/r002hs/r002hs.vtune",
        "last_path_csv": None,
        "last_path_db": "E:\\AAStudy\\0_work\\Work\\Chukanov\\MyWork\\Diplom\\GPTPROF\\openmw.db",
        "last_path_callstack": "filtered_2_3_r002hs_callstack_MWRender_IntersectionVisitorWithIgnoreList_skipTransform_5.csv",
        "module_names": [],
        "command": "",
        "args": "",
        "cleaned_args": "",
        "top_n": ""
    }

    try:
        # Проверяем, является ли команда "exit"
        if user_input.strip().lower() == "exit":
            logging.info("Exit command received. Terminating the session.")
            if gradio_interface_instance is not None:
                gradio_interface_instance.close()
            os._exit(0)

        # Логируем вызов графа
        logging.info("Invoking the graph with the current state.")
        result_state = compiled_graph.invoke(state)

        # Логируем результат выполнения графа
        logging.info(f"Graph execution completed. Result state: {result_state}")
        return result_state["result"], result_state  # Перезапись глобального состояния

    except Exception as e:
        # Логируем ошибку
        logging.error(f"An error occurred during graph execution: {e}")
        return f"Error: {str(e)}", state

def gradio_interface(user_input: str, session_state: dict = None):
    return ask_codestral(user_input, session_state)

def launch_gradio():
    global gradio_interface_instance

    if gradio_interface_instance is not None:
        gradio_interface_instance.close()

    with gr.Blocks() as interface:
        # Две колонки: левая для ввода команд, правая для вывода результатов
        with gr.Row():
            # Левая колонка: Command
            with gr.Column(scale=1):
                gr.Markdown("### Command Input")
                cmd_input = gr.Textbox(
                    label="Enter Command",
                    placeholder="e.g., show_db 7",
                    interactive=True,
                    lines=5
                )
                submit_btn = gr.Button("Submit", variant="primary")

            # Правая колонка: Output
            with gr.Column(scale=2):
                gr.Markdown("### Output")
                output = gr.Textbox(
                    label="Output",
                    value="Ready to process commands.",
                    lines=15,
                    interactive=False
                )

        # Состояние сессии
        session_state = gr.State()

        def process_command(cmd: str, state: dict):
            """
            Processes the user command and updates the Gradio interface.

            Args:
                cmd (str): User input command.
                state (dict): Current session state.

            Returns:
                tuple: Response message and updated state.
            """
            try:
                # Выполнение команды через граф
                response, new_state = gradio_interface(cmd, state)
                return response, new_state

            except Exception as e:
                # Обработка ошибок
                error_message = f"Error processing command: {e}"
                return error_message, state

        # Обработка нажатия кнопки и Enter в текстовом поле
        submit_btn.click(
            process_command,
            inputs=[cmd_input, session_state],
            outputs=[output, session_state]
        )
        cmd_input.submit(
            process_command,
            inputs=[cmd_input, session_state],
            outputs=[output, session_state]
        )

        # Инструкции для пользователя
        with gr.Row():
            gr.Markdown("""
            ### Available commands:
            - `export path/to/file.vtune` - Export VTune data to CSV (e.g. `export results.vtune`)
            - `filter module1,module2,...` - Filter data by specified modules
            - `show_db count_rows` - Show top N rows from the database
            - `exit` - Terminate the session
            """)

    gradio_interface_instance = interface

    # Настройки запуска
    interface.launch(
        server_port=7860,
        inbrowser=False,
        share=False,
        show_error=True
    )

In [ ]:
# Запускаем интерфейс
launch_gradio()